In [288]:
import os
import json
import pandas as pd
import traceback

In [289]:
from dotenv import load_dotenv
load_dotenv()

True

In [290]:
KEY = os.getenv("GOOGLEPALM_API_KEY")

In [291]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=KEY)

In [292]:
llm

ChatGoogleGenerativeAI(model='gemini-1.5-pro', google_api_key=SecretStr('**********'), client=genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [293]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [294]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [295]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given above the text, it is your job to \
create a quiz of {number} multiple questions for {subject} students in {tone} tone.
Make sure to format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [296]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number","subject","tone","response_json"],
    template=TEMPLATE
)

In [297]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt,output_key="quiz",verbose=True)

In [298]:
TEMPLATE2 = """
You are an expert grammerian and writer. Given a multiple choice question for {subject} students.\
you need to evaluate the complexicity of the question and give a complete analysis of the quiz. only use at max 50 words for complexicity
if the quiz is not at par with the cognitive and analytical abilities of the student,\
update the quiz questions which needs to be changed and the change the tone such that it perfectly fit the students ability.\
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [299]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [300]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [301]:
generate_evaluate_chain = SequentialChain(
    chains = [quiz_chain,review_chain],
    input_variables = ["text", "number","subject","tone","response_json"],
    output_variables = ["quiz","review"],
    verbose = True
)

In [302]:
file_path = 'D:\Projects\MCQ_Generator\data.txt'

In [303]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [304]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [305]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [306]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [307]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [308]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [309]:
print(type(response))

<class 'dict'>


In [310]:
quiz = response.get("quiz")

In [311]:
quiz_string = quiz.replace("```json", "").replace("```", "").strip()

In [312]:
print(quiz_string)

{
"1": {
"mcq": "Who first coined the term 'machine learning'?",
"options": {
"a": "Alan Turing",
"b": "Donald Hebb",
"c": "Arthur Samuel",
"d": "Tom M. Mitchell"
},
"correct": "c"
},
"2": {
"mcq": "Donald Hebb's work on interacting neurons laid the groundwork for what aspect of modern AI?",
"options": {
"a": "Computer vision",
"b": "Natural language processing",
"c": "Artificial neural networks",
"d": "Reinforcement learning"
},
"correct": "c"
},
"3": {
"mcq": "What was the primary function of the early 'learning machine' Cybertron?",
"options": {
"a": "Playing checkers",
"b": "Predicting stock prices",
"c": "Recognizing patterns in data",
"d": "Translating languages"
},
"correct": "c"
},
"4": {
"mcq": "Tom M. Mitchell's definition of machine learning emphasizes which key concept?",
"options": {
"a": "Human-like thought processes",
"b": "Improved performance with experience",
"c": "The ability to pass the Turing test",
"d": "The use of specific programming languages"
},
"correct": "b"

In [313]:
quiz = json.loads(quiz_string)

In [314]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

print(f"Number of items in quiz_table_data: {len(quiz_table_data)}")

Number of items in quiz_table_data: 5


In [315]:
quiz_table_data

[{'MCQ': "Who first coined the term 'machine learning'?",
  'Choices': 'a: Alan Turing | b: Donald Hebb | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': "Donald Hebb's work on interacting neurons laid the groundwork for what aspect of modern AI?",
  'Choices': 'a: Computer vision | b: Natural language processing | c: Artificial neural networks | d: Reinforcement learning',
  'Correct': 'c'},
 {'MCQ': "What was the primary function of the early 'learning machine' Cybertron?",
  'Choices': 'a: Playing checkers | b: Predicting stock prices | c: Recognizing patterns in data | d: Translating languages',
  'Correct': 'c'},
 {'MCQ': "Tom M. Mitchell's definition of machine learning emphasizes which key concept?",
  'Choices': 'a: Human-like thought processes | b: Improved performance with experience | c: The ability to pass the Turing test | d: The use of specific programming languages',
  'Correct': 'b'},
 {'MCQ': 'What are the two main objectives of modern machine learn

In [316]:
quiz = pd.DataFrame(quiz_table_data)

In [317]:
quiz.to_csv("MachineLearning.csv",index= False)